In [1]:
import os

try:
    # Running as normal Python script inside src/
    this_file = os.path.abspath(__file__)
    src_root = os.path.dirname(this_file)                        # EMOTION-PRED/src
    project_root = os.path.dirname(src_root)                    # EMOTION-PRED/
except NameError:
    # Running inside Jupyter (likely src/notebooks or src/)
    cwd = os.getcwd()

    # If running inside src/notebooks → go up one level
    if cwd.endswith("notebooks"):
        src_root = os.path.abspath(os.path.join(cwd, ".."))
        project_root = os.path.dirname(src_root)
    else:
        # Running from project root directly
        project_root = cwd
        src_root = os.path.join(project_root, "src")

# Final unified paths
results_root = os.path.join(src_root, "results")
data_root = os.path.join(src_root, "data","MAMS-ACSA","raw","data_jsonl")
print(f"📂 Project root: {project_root}"
      f"\n📂 Source root: {src_root}"
      f"\n📂 Results root: {results_root}"
      f"\n📂 Data root: {data_root}")
# 3 — JSONL files
TRAIN_JSONL = os.path.join(data_root, "train.jsonl")
VAL_JSONL   = os.path.join(data_root, "val.jsonl")
TEST_JSONL  = os.path.join(data_root, "test.jsonl")
SAMPLE_JSONL = os.path.join(data_root, "sample.jsonl")
print("Using dataset directory:", data_root)



📂 Project root: /Users/hd/Desktop/EMOTION-PRED
📂 Source root: /Users/hd/Desktop/EMOTION-PRED/src
📂 Results root: /Users/hd/Desktop/EMOTION-PRED/src/results
📂 Data root: /Users/hd/Desktop/EMOTION-PRED/src/data/MAMS-ACSA/raw/data_jsonl
Using dataset directory: /Users/hd/Desktop/EMOTION-PRED/src/data/MAMS-ACSA/raw/data_jsonl


In [2]:
GUIDELINES = """
==================================================
EMOTION-ONLY ANNOTATION GUIDELINES (V2 – FINAL)
==================================================

You are performing EMOTION-ONLY annotation.

Aspect and polarity are GIVEN and MUST NOT be changed.
Your task is to select exactly ONE emotion from the
allowed_emotions list provided in the task.

The allowed_emotions list is authoritative.
Do NOT invent or substitute emotion labels.

--------------------------------------------------
ASPECT DEFINITIONS & CUES
--------------------------------------------------

AMBIENCE
• Atmosphere or sensory environment.
• Includes: decor, lighting, music, noise, cleanliness, comfort, vibe.
• Cues: atmosphere, vibe, cozy, romantic, noisy, crowded, dirty.
• Positive: lovely atmosphere.
• Negative: too loud, dirty tables.
• Neutral: dim lighting.

FOOD
• Taste, flavor, appearance, freshness, food quality.
• Cues: delicious, bland, fresh, overcooked, gross.
• Positive: delicious pizza.
• Negative: dry meat, awful soup.
• Neutral: factual description of dishes.

MENU
• Menu variety, availability, layout, options.
• Cues: variety, selection, limited, unavailable, confusing menu.
• Positive: great variety.
• Negative: confusing or limited menu.
• Neutral: standard menu listing.

PLACE
• Physical location or spatial setup.
• Includes: accessibility, parking, crowding, space size.
• Cues: location, parking, cramped, hard to find.
• Positive: perfect location.
• Negative: cramped space.
• Neutral: near the station.

PRICE
• Cost and value for money.
• Cues: expensive, overpriced, cheap, worth it, rip-off.
• Positive: good value.
• Negative: too expensive.
• Neutral: price stated without judgment.

SERVICE
• Operational process, NOT people.
• Includes: speed, wait time, order accuracy, reservations, payment.
• Cues: slow service, long wait, efficient service.
• Positive: fast service.
• Negative: long waiting time.
• Neutral: counter service.

STAFF
• Human behavior and attitude.
• Includes: friendliness, rudeness, professionalism, attentiveness.
• Cues: waiter, server, rude, friendly, ignored.
• Positive: friendly staff.
• Negative: rude waiter.
• Neutral: staff uniforms mentioned.

MISCELLANEOUS
• Overall impressions not tied to a specific aspect.
• Cues: overall experience, visit, general impression.
• Positive: wonderful time.
• Negative: terrible experience.
• Neutral: popular restaurant.

--------------------------------------------------
EMOTION DEFINITIONS (SEMANTIC GUIDANCE)
--------------------------------------------------

The following definitions explain how to interpret emotions.
You MUST still choose from the allowed_emotions list.

ADMIRATION
• Strong positive emotion due to exceptional or impressive quality.
• Requires elevated praise.
• Cues: amazing, outstanding, excellent, impressive, top-notch.
• NOT admiration: nice, good, enjoyable, liked it.

SATISFACTION
• Enjoyment, liking, comfort, expectations met.
• Cues: enjoyed, happy, pleasant, liked, good, relaxing.
• If positive but NOT exceptional → choose SATISFACTION.

GRATITUDE
• Thankfulness for a specific helpful action by staff.
• Cues: thank you, appreciated, grateful.

ANNOYANCE
• Mild to moderate irritation or inconvenience.
• Cues: annoying, frustrating, noisy, slow, inconvenient.

DISAPPOINTMENT
• Unmet expectations or let-down.
• Cues: disappointed, expected more, unfortunately, let down.

DISGUST
• Strong rejection or revulsion.
• Cues: disgusting, gross, filthy, unsafe, made me sick.

NO_EMOTION
• Factual mention without emotional reaction.
• Use when no opinion or feeling is expressed.

--------------------------------------------------
META LABELS (ANNOTATION SEMANTICS)
--------------------------------------------------

MENTIONED_ONLY
• Aspect is referenced only as background information.
• No opinion or emotional reaction is expressed.
• Use ONLY if it appears in allowed_emotions.

MIXED_EMOTIONS
• Conflicting emotions are expressed for the SAME aspect.
• Use ONLY if it appears in allowed_emotions.

--------------------------------------------------
CRITICAL PRIORITY RULES
--------------------------------------------------

1. Always choose from allowed_emotions.
2. If text expresses liking or enjoyment WITHOUT exceptional praise → prefer SATISFACTION (if allowed).
3. Use ADMIRATION only for exceptional or impressive quality.
4. Mentioned_only and Mixed_emotions are NOT emotions, but annotation states.
5. If uncertain → choose the most neutral option available in allowed_emotions.
6. Follow definitions strictly, even if another label sounds more expressive.

==================================================
"""

In [ ]:
import json
import os
import re
import time
import requests
from dotenv import load_dotenv

# ======================================================
# API SETUP
# ======================================================
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

MODEL = "models/gemini-2.5-flash"
URL = f"https://generativelanguage.googleapis.com/v1beta/{MODEL}:generateContent"
HEADERS = {
    "Content-Type": "application/json",
    "X-goog-api-key": API_KEY
}

# ======================================================
# PATHS
# ======================================================
IN_PATH = os.path.join(data_root, "daniel_50.jsonl")
EMOTION_JSON = os.path.join(data_root, "emotion.json")

# ---- derive input identifier ----
input_name = os.path.splitext(os.path.basename(IN_PATH))[0]
OUT_DIR = os.path.join(results_root, f"{input_name}_emotion_v2")
os.makedirs(OUT_DIR, exist_ok=True)

OUT_CLEAN = os.path.join(OUT_DIR, f"gemini_emotion_only_v2_{input_name}.jsonl")
OUT_REASON = os.path.join(OUT_DIR, f"gemini_emotion_only_v2_reasons_{input_name}.jsonl")

# ======================================================
# LOAD DATA
# ======================================================
EMOTIONS = json.load(open(EMOTION_JSON, "r", encoding="utf-8"))
rows = [json.loads(line) for line in open(IN_PATH, "r", encoding="utf-8")]

# ======================================================
# NORMALIZE POLARITY
# ======================================================
def normalize_pol(pol):
    pol = pol.lower().strip()
    return {"neagative": "negative", "postive": "positive", "nutral": "neutral"}.get(pol, pol)

# ======================================================
# GEMINI CALL
# ======================================================
def ask_gemini(prompt):
    payload = {"contents": [{"parts": [{"text": prompt}]}]}
    while True:
        try:
            r = requests.post(URL, headers=HEADERS, json=payload)
            r.raise_for_status()
            return r.json()["candidates"][0]["content"]["parts"][0]["text"]
        except Exception as e:
            print("⚠️ Gemini error → retrying:", e)
            time.sleep(3)

# ======================================================
# JSON PARSER
# ======================================================
def parse_json(txt):
    if not txt:
        return None
    txt = txt.replace("```json", "").replace("```", "").strip()
    txt = re.sub(r",\s*}", "}", txt)
    txt = re.sub(r",\s*]", "]", txt)
    try:
        return json.loads(txt)
    except:
        return None

# ======================================================
# PROMPT BUILDER
# ======================================================
def build_prompt(review, triples):

    block = ""
    for i, t in enumerate(triples):
        asp = t["aspect"]
        pol = normalize_pol(t["polarity"])
        allowed = EMOTIONS[asp][pol]
        block += f"""
{i+1}. aspect: {asp}
   polarity: {pol}
   allowed_emotions: {allowed}
"""

    return f"""
You are performing EMOTION-ONLY annotation.

Follow the rules EXACTLY.

{GUIDELINES}

### Review:
\"""{review}\"""

### Triples:
{block}

### Return STRICT JSON:
[
  {{
    "emotion": "...",
    "reason": "Short explanation referring to the definition used."
  }}
]

Rules:
- JSON only
- One object per triple
- Emotion must be chosen from allowed_emotions
- Reason must justify the chosen emotion
- If unsure → choose No Emotion
"""

# ======================================================
# MAIN LOOP
# ======================================================
with open(OUT_CLEAN, "w", encoding="utf-8") as f_clean, \
     open(OUT_REASON, "w", encoding="utf-8") as f_reason:

    total = len(rows)

    for idx, row in enumerate(rows, start=1):
        review = row["input"]
        triples = row["output"]

        prompt = build_prompt(review, triples)

        while True:
            reply = ask_gemini(prompt)
            parsed = parse_json(reply)

            if isinstance(parsed, list) and len(parsed) == len(triples):
                valid = True
                for out, t in zip(parsed, triples):
                    asp = t["aspect"]
                    pol = normalize_pol(t["polarity"])
                    allowed = EMOTIONS[asp][pol]
                    emo = out.get("emotion", "").strip().capitalize()
                    if emo not in allowed:
                        valid = False
                        break
                if valid:
                    break

            print(f"❌ Invalid output at row {idx}, retrying…")
            time.sleep(2)

        clean_out = []
        reason_out = []

        for t, out in zip(triples, parsed):
            emo = out["emotion"].strip().capitalize()
            reason = out.get("reason", "").strip()

            clean_out.append({
                "aspect": t["aspect"],
                "polarity": normalize_pol(t["polarity"]),
                "emotion": emo
            })

            reason_out.append({
                "aspect": t["aspect"],
                "polarity": normalize_pol(t["polarity"]),
                "emotion": emo,
                "reason": reason
            })

        f_clean.write(json.dumps({"input": review, "output": clean_out}, ensure_ascii=False) + "\n")
        f_reason.write(json.dumps({"input": review, "details": reason_out}, ensure_ascii=False) + "\n")

        print(f"✔ [{idx}/{total}] processed")

print("\nDONE")
print("CLEAN →", OUT_CLEAN)
print("REASONS →", OUT_REASON)

❌ Invalid output at row 1, retrying…
❌ Invalid output at row 1, retrying…


KeyboardInterrupt: 

In [ ]:
# import json
# import os
# import re
# import time
# import requests
# from dotenv import load_dotenv

# # ======================================================
# # API SETUP
# # ======================================================
# load_dotenv()
# API_KEY = os.getenv("GEMINI_API_KEY")

# MODEL = "models/gemini-2.5-flash"
# URL = f"https://generativelanguage.googleapis.com/v1beta/{MODEL}:generateContent"
# HEADERS = {
#     "Content-Type": "application/json",
#     "X-goog-api-key": API_KEY
# }

# # ======================================================
# # PATHS
# # ======================================================
# IN_PATH = os.path.join(data_root, "cleaned_300.jsonl")
# EMOTION_JSON = os.path.join(data_root, "emotion.json")

# OUT_DIR = os.path.join(results_root, "emotion-minimal")
# os.makedirs(OUT_DIR, exist_ok=True)

# OUT_JSON = os.path.join(OUT_DIR, "emotion_only_cleaned_300.jsonl")

# # ======================================================
# # LOAD DATA
# # ======================================================
# EMOTIONS = json.load(open(EMOTION_JSON, "r", encoding="utf-8"))
# rows = [json.loads(line) for line in open(IN_PATH, "r", encoding="utf-8")]

# # ======================================================
# # NORMALIZE POLARITY
# # ======================================================
# def normalize_pol(pol):
#     pol = pol.lower().strip()
#     mapping = {"neagative": "negative", "postive": "positive", "nutral": "neutral"}
#     return mapping.get(pol, pol)


# # ======================================================
# # GEMINI CALL (retry until success)
# # ======================================================
# def ask_gemini(prompt):
#     payload = {"contents": [{"parts": [{"text": prompt}]}]}

#     while True:
#         start = time.time()
#         try:
#             r = requests.post(URL, headers=HEADERS, json=payload)
#             r.raise_for_status()
#             elapsed = time.time() - start
#             return r.json()["candidates"][0]["content"]["parts"][0]["text"], elapsed
#         except Exception as e:
#             print(f"⚠️ Gemini error → retrying in 3s: {e}")
#             time.sleep(3)


# # ======================================================
# # JSON PARSER
# # ======================================================
# def try_parse_json(txt):
#     if txt is None:
#         return None
#     cleaned = txt.strip()
#     cleaned = cleaned.replace("```json", "").replace("```", "").strip()
#     cleaned = re.sub(r",\s*}", "}", cleaned)
#     cleaned = re.sub(r",\s*]", "]", cleaned)
#     try:
#         return json.loads(cleaned)
#     except:
#         return None


# # ======================================================
# # BUILD MINIMAL EMOTION-ONLY PROMPT
# # ======================================================
# def build_prompt(review, triples):

#     block = ""
#     for i, t in enumerate(triples):
#         asp = t["aspect"]
#         pol = normalize_pol(t["polarity"])
#         allowed = EMOTIONS[asp][pol]
#         block += f"""
# {i+1}. aspect: {asp}
#    polarity: {pol}
#    allowed_emotions: {allowed}
# """

#     return f"""
# Predict ONLY the emotions for each (aspect, polarity) pair.

# ### Review:
# \"""{review}\"""

# ### Triples:
# {block}

# ### Return STRICT JSON:
# [
#   {{"emotion": "..."}},
#   ...
# ]

# Rules:
# - JSON only
# - Number of items must match the number of triples
# - Emotion must be chosen from allowed_emotions
# """


# # ======================================================
# # MAIN LOOP — WRITE EACH RESULT IMMEDIATELY
# # ======================================================
# f_out = open(OUT_JSON, "a", encoding="utf-8")
# total = len(rows)

# for idx, row in enumerate(rows, start=1):
#     review = row["input"]
#     triples = row["output"]

#     prompt = build_prompt(review, triples)

#     # retry until JSON is correct
#     while True:
#         reply, elapsed = ask_gemini(prompt)
#         parsed = try_parse_json(reply)

#         if isinstance(parsed, list) and len(parsed) == len(triples):
#             valid = True
#             for out, t in zip(parsed, triples):
#                 asp = t["aspect"]
#                 pol = normalize_pol(t["polarity"])
#                 allowed = EMOTIONS[asp][pol]
#                 emo = out.get("emotion", "").strip().capitalize()
#                 if emo not in allowed:
#                     valid = False
#                     break

#             if valid:
#                 break

#         print(f"❌ Invalid JSON or structure at row {idx}. Retrying…")
#         time.sleep(2)

#     # Build final clean output
#     final_objs = []
#     for t, out in zip(triples, parsed):
#         asp = t["aspect"]
#         pol = normalize_pol(t["polarity"])
#         emo = out["emotion"].strip().capitalize()
#         final_objs.append({
#             "aspect": asp,
#             "polarity": pol,
#             "emotion": emo
#         })

#     # Write IMMEDIATELY
#     f_out.write(json.dumps({"input": review, "output": final_objs}, ensure_ascii=False) + "\n")

#     print(f"✔ [{idx}/{total}] OK — {len(triples)} emotions — {elapsed:.1f}s")

# f_out.close()
# print("\nDONE →", OUT_JSON)